<b><h2>Reconocimiento de Entidades Nombradas utilizando NLTK y SpaCy</h2></b>

El reconocimiento de entidades nombradas es probablemente el primer paso hacia la extracción de información que busca localizar entidades nombradas que se encuentran dentro del texto y clasificarlas en categorías predefinidas tales como personas, organizaciones, lugares, expresiones de tiempo (fechas), cantidades, valores monetarios, porcentajes, etc.

El reconocimiento de entidades nombradas tiene muchas aplicaciones dentro del procesamiento de lenguaje natural y puede ayudar a resolver muchas preguntas como:

<ul>
<li>Qué compañias son mencionadas en un artículo o noticia
<li>La mención de productos específicos en alguna reseña o comentario
<li>El nombre de personas y su localización
</ul>

Pues bien, como es costumbre, comenzaremos por importar las librerías necesarias. Comenzaremos trabajando con NLTK y posteriormente trabajaremos con SpaCy

In [1]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

In [2]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jumunoz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\jumunoz\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\jumunoz\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\jumunoz\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping help\tagsets.zip.


True

Una vez que se han cargado las librerías, se toma un texto para comenzar a probar. pare este ejemplo se ha tomado un texto obtenido de una noticia.

'European authorities fined Google a record $5.1 billion on Wednesday for abusing its power in the mobile phone market and ordered the company to alter its practices'

In [3]:
text = 'European authorities fined Google a record $5.1 billion on Wednesday for abusing its power in the mobile phone market and ordered the company to alter its practices'

Se aplica tokenization y part-of-speech para generar etiquetas para las palabras en la oración. Para ello se define una función que recibe como parámetro el enunciado (sent) y le aplica ambos métodos.

El resultado de esta función muestra una lista de palabras (tokens) con su clasificación.

In [4]:
def preprocess(sent):
    sent = nltk.word_tokenize(sent)
    sent = nltk.pos_tag(sent)
    return sent

In [5]:
sent = preprocess(text)
sent

[('European', 'JJ'),
 ('authorities', 'NNS'),
 ('fined', 'VBD'),
 ('Google', 'NNP'),
 ('a', 'DT'),
 ('record', 'NN'),
 ('$', '$'),
 ('5.1', 'CD'),
 ('billion', 'CD'),
 ('on', 'IN'),
 ('Wednesday', 'NNP'),
 ('for', 'IN'),
 ('abusing', 'VBG'),
 ('its', 'PRP$'),
 ('power', 'NN'),
 ('in', 'IN'),
 ('the', 'DT'),
 ('mobile', 'JJ'),
 ('phone', 'NN'),
 ('market', 'NN'),
 ('and', 'CC'),
 ('ordered', 'VBD'),
 ('the', 'DT'),
 ('company', 'NN'),
 ('to', 'TO'),
 ('alter', 'VB'),
 ('its', 'PRP$'),
 ('practices', 'NNS')]

In [6]:
nltk.help.upenn_tagset('NNS')

NNS: noun, common, plural
    undergraduates scotches bric-a-brac products bodyguards facets coasts
    divestitures storehouses designs clubs fragrances averages
    subjectivists apprehensions muses factory-jobs ...


https://www.learntek.org/blog/categorizing-pos-tagging-nltk-python/

Lo que sigue es agrupar las frases nominales (grupo de palabras cuyo núcleo está constituido por un sustantivo) para identificar entidades nombradas. Para ello utilizaremos una expresión regular que define las reglas que indican cómo se deben obtener los fragmentos de una oración.

Ejemplo de frase nominal El hijo de la vecina es muy inteligente.

El patron para formar fragmentos consiste en una regla que establece que una frase nominal (NP) debe formarse cada vez que se encuentre un <a href="https://grammar.yourdictionary.com/parts-of-speech/nouns/what/what-is-a-determiner.html">determinante opcional</a> (DT) seguido de cualquier número de adjetivos (JJ), y por último un sustantivo (NN).

In [7]:
pattern = 'NP: {<DT>?<JJ>*<NN>}'

Utilizando este patrón se crea un parser de fragmentos y se prueba con el enunciado de ejemplo.
En la impresión que se genera como resultado podemos ver que, siguiendo el patrón establecido se forman fragmentos como:

(NP a/DT record/NN)
(NP power/NN)
(NP the/DT mobile/JJ phone/NN)
(NP the/DT company/NN)

Los cuales están conformados por los elementos establecidos en la expresión regular.

In [8]:
cp = nltk.RegexpParser(pattern)
cs = cp.parse(sent)
print(cs)

(S
  European/JJ
  authorities/NNS
  fined/VBD
  Google/NNP
  (NP a/DT record/NN)
  $/$
  5.1/CD
  billion/CD
  on/IN
  Wednesday/NNP
  for/IN
  abusing/VBG
  its/PRP$
  (NP power/NN)
  in/IN
  (NP the/DT mobile/JJ phone/NN)
  (NP market/NN)
  and/CC
  ordered/VBD
  (NP the/DT company/NN)
  to/TO
  alter/VB
  its/PRP$
  practices/NNS)


<b><h1>Etiquetas IOB</h1></b>

Las <a href="https://www.geeksforgeeks.org/nlp-iob-tags/">Etiquetas IOB</a> se han convertido en una forma estandar de representar fragmentos de texto.

Estas etiquetas son similares a las que se obtienen por medio de part-of-speech pero pueden decir el inicio, interior y exterior de un fragmento. No solo permiten frases nominales sino también otro tipo de frases.

En el siguiente ejemplo se importan las librerías para la obtención de estas etiquetas.

In [20]:
from nltk.chunk import conlltags2tree, tree2conlltags
from pprint import pprint
iob_tagged = tree2conlltags(cs)
pprint(iob_tagged)

[('European', 'JJ', 'O'),
 ('authorities', 'NNS', 'O'),
 ('fined', 'VBD', 'O'),
 ('Google', 'NNP', 'O'),
 ('a', 'DT', 'B-NP'),
 ('record', 'NN', 'I-NP'),
 ('$', '$', 'O'),
 ('5.1', 'CD', 'O'),
 ('billion', 'CD', 'O'),
 ('on', 'IN', 'O'),
 ('Wednesday', 'NNP', 'O'),
 ('for', 'IN', 'O'),
 ('abusing', 'VBG', 'O'),
 ('its', 'PRP$', 'O'),
 ('power', 'NN', 'B-NP'),
 ('in', 'IN', 'O'),
 ('the', 'DT', 'B-NP'),
 ('mobile', 'JJ', 'I-NP'),
 ('phone', 'NN', 'I-NP'),
 ('market', 'NN', 'B-NP'),
 ('and', 'CC', 'O'),
 ('ordered', 'VBD', 'O'),
 ('the', 'DT', 'B-NP'),
 ('company', 'NN', 'I-NP'),
 ('to', 'TO', 'O'),
 ('alter', 'VB', 'O'),
 ('its', 'PRP$', 'O'),
 ('practices', 'NNS', 'O')]


En esta resultado se puede ver que hay un token por cada línea, cada uno con su etiqueta de part-of-speech y su etiqueta de entidad nombrada.

Teniendo como base este corpus de entrenamiento, se puede construir un etiquetador que puede ser usado para etiquetar nuevos enunciados, y utilizar la función nltk.chunk.conlltags2tree() para convertir la secuencia de etiquetas en un árbol de fragmentos.

Con la función nltk.ne_chunk() se pueden reconocer entidades nombradas utilizando un clasificador que agregue etiquetas de categoria como PERSON, ORGANIZATION, GPE (Geo-Political Entities (ciudades, estados, provincias, países)), etc.

In [9]:
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\jumunoz\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping chunkers\maxent_ne_chunker.zip.
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\jumunoz\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\words.zip.


True

In [10]:
ne_tree = nltk.ne_chunk(pos_tag(word_tokenize(text)))
print(ne_tree)

(S
  (GPE European/JJ)
  authorities/NNS
  fined/VBD
  (PERSON Google/NNP)
  a/DT
  record/NN
  $/$
  5.1/CD
  billion/CD
  on/IN
  Wednesday/NNP
  for/IN
  abusing/VBG
  its/PRP$
  power/NN
  in/IN
  the/DT
  mobile/JJ
  phone/NN
  market/NN
  and/CC
  ordered/VBD
  the/DT
  company/NN
  to/TO
  alter/VB
  its/PRP$
  practices/NNS)


<b><h1>SpaCy</h1></b>

El reconocimiento de entidades utilizando la librería SpaCy ha sido entrenado utilizando el corpus de <a href="https://catalog.ldc.upenn.edu/LDC2013T19"> OntoNotes5</a> y soporta las siguientes <a href="https://spacy.io/api/annotation#section-named-entities"> entidades</a>:

In [12]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

Para este ejemplo vamos a usar el mismo enunciado.

Una de las cosas más interesantes de SpaCy es que es un poco más simple.

El enunciado se envía como parámetro a la función nlp() y posteriormente se puede imprimir como resultado la clasificación de cada entidad nombrada que reconozca.

In [13]:
doc = nlp('European authorities fined Google a record $5.1 billion on Wednesday for abusing its power in the mobile phone market and ordered the company to alter its practices')
print([(X.text, X.label_) for X in doc.ents])

[('European', 'NORP'), ('Google', 'ORG'), ('$5.1 billion', 'MONEY'), ('Wednesday', 'DATE')]


Se puede ver a diferencia del ejemplo anterior que tiene mayor precisión al momento de reconocer entidades, ya que por ejemplo 'European' la ha clasificado como NORP (nationalities or religious or political groups) y Google la ha clasificado como organization y no como PERSON.

<b><h1>Tokens</h1></b>

Hasta ahora hemos trabajado con reconocimiento a nivel entidad. En el siguiente ejemplo vamos a trabajar con reconocimiento a nivel token utilizando el esquema de etiquetado <a href="https://spacy.io/api/annotation#section-named-entities"> BILUO</a> para describir fronteras de entidades.

La "B" significa que el token comienza una entidad, la "I" siginfica que el token está dentro de una entidad o forma parte de una entidad, la "O" significa que el token está por fuera de la entidad, y finalmente el espacio vacío "" significa que no se pudo establecer una etiqueta para la entidad.

In [14]:
print([(X, X.ent_iob_, X.ent_type_) for X in doc])

[(European, 'B', 'NORP'), (authorities, 'O', ''), (fined, 'O', ''), (Google, 'B', 'ORG'), (a, 'O', ''), (record, 'O', ''), ($, 'B', 'MONEY'), (5.1, 'I', 'MONEY'), (billion, 'I', 'MONEY'), (on, 'O', ''), (Wednesday, 'B', 'DATE'), (for, 'O', ''), (abusing, 'O', ''), (its, 'O', ''), (power, 'O', ''), (in, 'O', ''), (the, 'O', ''), (mobile, 'O', ''), (phone, 'O', ''), (market, 'O', ''), (and, 'O', ''), (ordered, 'O', ''), (the, 'O', ''), (company, 'O', ''), (to, 'O', ''), (alter, 'O', ''), (its, 'O', ''), (practices, 'O', '')]


# **En español**


In [15]:
!python -m spacy download es_core_news_md

     ---------------------------------------- 0.0/42.3 MB ? eta -:--:--
     --------------------------------------- 0.0/42.3 MB 330.3 kB/s eta 0:02:08
     --------------------------------------- 0.1/42.3 MB 550.5 kB/s eta 0:01:17
     ---------------------------------------- 0.2/42.3 MB 1.2 MB/s eta 0:00:36
     ---------------------------------------- 0.4/42.3 MB 1.9 MB/s eta 0:00:22
      --------------------------------------- 0.6/42.3 MB 2.7 MB/s eta 0:00:16
     - -------------------------------------- 1.3/42.3 MB 4.7 MB/s eta 0:00:09
     - -------------------------------------- 1.9/42.3 MB 5.9 MB/s eta 0:00:07
     -- ------------------------------------- 2.2/42.3 MB 5.8 MB/s eta 0:00:07
     --- ------------------------------------ 3.6/42.3 MB 8.6 MB/s eta 0:00:05
     ---- ----------------------------------- 4.4/42.3 MB 9.4 MB/s eta 0:00:05
     ---- ----------------------------------- 4.7/42.3 MB 9.6 MB/s eta 0:00:04
     ----- ---------------------------------- 5.4/42.3 MB

In [16]:
import spacy
import es_core_news_md
nlp = es_core_news_md.load()

In [17]:
from spacy import displacy

In [18]:
texto = nlp("Carlos Slim Helú (Ciudad de México, 28 de enero de 1940) es un empresario e ingeniero mexicano. Es el decimocuarto hombre más rico del mundo, ya que posee bienes que ascienden a los 102 800 millones de dólares, lo cual representa alrededor del 6% del producto interno bruto de México y lo convierte en la persona más rica de México y de América Latina. Entre 2010 y 2013 fue la persona más rica del mundo según la revista Forbes.")

In [19]:
for sent in texto.sents:
  displacy.render(nlp(sent.text), jupyter=True, style='ent')